# Pre-processing scRNA-seq data from two Parkinson's disease studies

## Description

* This notebook covers functions to pre-process data from single nuclei RNA-seq (snRNA-seq) data from two studies [Agarwal_2020](https://pubmed.ncbi.nlm.nih.gov/32826893/) and [Kamath_2022](https://pubmed.ncbi.nlm.nih.gov/35513515/).     
    * The Agarwal_2020 dataset was obtained from [GEO](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE140231) in `mtx` format and associated sample metadata. Cell type annotations were obtained from authors paper Table S2.
    * The Kamath_2022 dataset was obtained from [CELLxGENE](https://cellxgene.cziscience.com/collections/b0f0b447-ac37-45b0-b1bf-5c0b7d871120) in `h5ad` format with sample metadata and cell type annotations
    * A copy of these files can be downloaded from https://zenodo.org/uploads/15723900<br><br>

* It runs the following steps:
    * Load data from original sources (GEO and CELLxGENE)
    * Standardize sample metadata
    * Downsample ~103k cells from Kamath 2022 dataset. Keep all ~18k cells from Agarwal 2020.
    * Saves `zarr` and `h5ad` files for script `02_integration.ipynb`.<br><br>


* h5ad vs. zarr formats:
    * For the downsampled Kamath 2022 subset (~103k cells × ~34k genes), it takes the following disk space and speed to read:
        * `zarr`: size=921M, read time = 5.2s `adata.write_zarr(..., chunks=[adata.shape[0], 5])`
        * `h5ad` compressed: size=929M , read time = 8.4s `adata.write(..., compression='gzip')`
        * `h5ad` not compressed: size=3.1G, read time = 16.6s

In [1]:
### External modules
import warnings
import os
import scanpy as sc
import pandas as pd
import logging
import sys
from scipy.io import mmread
import zarr ### tested with zarr==2.18.7 and dask==2025.4.0

import io
import requests

### Internal modules
sys.path.append(os.path.expanduser("~/sc_and_spatial_parkinson/src/python"))
from utils.reformat_to_adata import convert_mtx_to_adata
from utils.add_metadata_to_adata import add_metadata_to_adata
from utils.reformat_to_adata import convert_zarr_to_adata

### Logs and warning settings
warnings.filterwarnings("ignore")
# logging.basicConfig(level=logging.DEBUG, format="%(asctime)s %(name)s %(levelname)s:\n%(message)s")
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)s %(levelname)s:\n%(message)s")
logger = logging.getLogger(__name__)

/Users/jdime/venv_bioinfo/lib/python3.13/site-packages/anndata/utils.py:434: FutureWarning: Importing read_elem from `anndata.experimental` is deprecated. Import anndata.io.read_elem instead.
  warnings.warn(msg, FutureWarning)


## Define inputs/outputs

In [2]:
### Sample metadata
google_id_sample_metadata = "1761mSSr1jY_SkJ9UvZ4Zyf1G39B9y0CGnzlItFEF9iY"
default_sample_metadata_cols = ['barcode', 'study_name', 'sample_name']
extra_sample_metadata_cols   = ['sample_source','sample_organism','sample_tissue_region',
                                'donor_gender','donor_age_years','cause_of_death',
                                'authors_main_cell_type','authors_sub_cell_type','Nurr_status']

### base outdir for zarr files
outdir = os.path.expanduser('~/sc_and_spatial_parkinson/data/')
outdir_zarr = os.path.join(outdir, 'zarr')
os.makedirs(outdir_zarr, exist_ok=True)

### measurements can be downloaded from https://zenodo.org/uploads/15723900
indir = os.path.expanduser('~/sc_and_spatial_parkinson/data/')
mtx_dir_agarwal = os.path.join(indir, 'original/Agarwal_NatCommun_2020_32826893/FROM_GEO/ORIGINAL_FILES')
ct_dir_agarwal = os.path.join(indir, 'original/Agarwal_NatCommun_2020_32826893/FROM_AUTHORS_PAPER')
h5ad_dir_kamath = os.path.join(indir, 'original/Kamath_NatNeurosci_2022_35513515/FROM_CELLXGENE/ORIGINAL_FILES')

### default parameters
default_missing = 'Unknown'

## Standardize Agarwal_2020 (from GEO mtx files)

#### Study parameters

In [3]:
sample_names = ['GSM4157067_Sample_5_C3', 'GSM4157068_Sample_6_N3', 'GSM4157069_Sample_7_N4', 'GSM4157070_Sample_8_N5',
                'GSM4157071_Sample_9_C1B', 'GSM4157072_Sample_10_N1B', 'GSM4157073_Sample_11_C2B', 'GSM4157074_Sample_12_N2B',
                'GSM4157075_Sample_13_C4B', 'GSM4157076_Sample_14_N4B', 'GSM4157077_Sample_15_C5B', 'GSM4157078_Sample_16_N5B',]

#### Convert each sample mtx into adata and concatenate into study adata

In [4]:
study_adatas = []
counter=0
study_name = "Agarwal_NatCommun_2020_32826893"
logger.info((f"***{study_name} Convert each sample mtx into adata and concatenate into study adata***"))

for sample_name in sample_names:
    counter = counter + 1
    logger.info((f"Load {counter}/{len(sample_names)} {sample_name} "))
    root_fp = os.path.join(mtx_dir_agarwal, sample_name)
    matrix_fp = os.path.join(root_fp, "matrix.mtx.gz")
    barcodes_fp = os.path.join(root_fp, "barcodes.tsv.gz")
    features_fp = os.path.join(root_fp, "genes.tsv.gz")
    
    matrix = mmread(matrix_fp) ### using scipy.io.mmread because it's faster than sc.read_10x_mtx
    barcodes_df = pd.read_csv(barcodes_fp, sep='\t', names=["barcode"])
    features_df = pd.read_csv(features_fp, sep='\t', usecols=[0, 1], index_col=0, names=["index", "gene"])
    
    sample_adata = convert_mtx_to_adata(
        matrix=matrix, barcodes_df=barcodes_df, features_df=features_df,
        barcodes_prefix=None
    )

    # Add fixed metadata
    sample_adata.obs.loc[:, 'barcode'] = sample_name + '_' + sample_adata.obs['barcode']
    sample_adata.obs.loc[:, 'study_name'] = study_name
    sample_adata.obs.loc[:, 'sample_name'] = sample_name
    sample_adata.obs.loc[:, 'cause_of_death'] = default_missing
    sample_adata.obs.loc[:, 'authors_main_cell_type'] = default_missing # populated below
    sample_adata.obs.loc[:, 'authors_sub_cell_type'] = default_missing # populated out below
    sample_adata.obs.loc[:, 'Nurr_status']    = default_missing

    # Uppercase gene names
    sample_adata.var['gene'] = sample_adata.var['gene'].apply(lambda x: x.upper())

    # Append to study adata
    study_adatas.append(sample_adata)
    if len(study_adatas) == 1:
        study_adata = study_adatas[0]
    else:
        study_adata = study_adatas[0].concatenate(study_adatas[1:], join="inner", index_unique=None)

INFO:__main__:***Agarwal_NatCommun_2020_32826893 Convert each sample mtx into adata and concatenate into study adata***
INFO:__main__:Load 1/12 GSM4157067_Sample_5_C3 
INFO:__main__:Load 2/12 GSM4157068_Sample_6_N3 
INFO:__main__:Load 3/12 GSM4157069_Sample_7_N4 
INFO:__main__:Load 4/12 GSM4157070_Sample_8_N5 
INFO:__main__:Load 5/12 GSM4157071_Sample_9_C1B 
INFO:__main__:Load 6/12 GSM4157072_Sample_10_N1B 
INFO:__main__:Load 7/12 GSM4157073_Sample_11_C2B 
INFO:__main__:Load 8/12 GSM4157074_Sample_12_N2B 
INFO:__main__:Load 9/12 GSM4157075_Sample_13_C4B 
INFO:__main__:Load 10/12 GSM4157076_Sample_14_N4B 
INFO:__main__:Load 11/12 GSM4157077_Sample_15_C5B 
INFO:__main__:Load 12/12 GSM4157078_Sample_16_N5B 


#### Add sample metadata

In [5]:
logger.info((f"***Add sample metadata***"))
input_sm = (f"https://docs.google.com/spreadsheets/d/{google_id_sample_metadata}/gviz/tq?tqx=out:csv&sheet={study_name}")
study_df = pd.read_csv(io.BytesIO(requests.get(input_sm).content), dtype=str)
study_df.info(verbose=False)

study_adata = add_metadata_to_adata(
    adata=study_adata,
    metadata_df=study_df,
    join=["study_name", "sample_name"],
    columns_to_add=extra_sample_metadata_cols,
)
study_adata.obs = study_adata.obs[default_sample_metadata_cols + extra_sample_metadata_cols]
logger.info((study_adata))

INFO:__main__:***Add sample metadata***
INFO:root:Adding metadata to adata, joining on ['study_name', 'sample_name'], keeping columns ['sample_source', 'sample_organism', 'sample_tissue_region', 'donor_gender', 'donor_age_years', 'cause_of_death', 'authors_main_cell_type', 'authors_sub_cell_type', 'Nurr_status']...
INFO:__main__:AnnData object with n_obs × n_vars = 18120 × 33694
    obs: 'barcode', 'study_name', 'sample_name', 'sample_source', 'sample_organism', 'sample_tissue_region', 'donor_gender', 'donor_age_years', 'cause_of_death', 'authors_main_cell_type', 'authors_sub_cell_type', 'Nurr_status'
    var: 'gene'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Columns: 12 entries, study_name to sample_biomaterial_provider
dtypes: object(12)
memory usage: 1.3+ KB


#### Add cell metadata

In [6]:
ada_ct = pd.read_excel(os.path.join(ct_dir_agarwal, 'Table_S2_cell_type_annotations.xlsx'))
ada_sample_dic = {
    'C5B': 'GSM4157077_Sample_15_C5B',
    'C3': 'GSM4157067_Sample_5_C3',
    'C2B': 'GSM4157073_Sample_11_C2B',
    'C4B': 'GSM4157075_Sample_13_C4B',
    'N3': 'GSM4157068_Sample_6_N3',
    'N2B': 'GSM4157074_Sample_12_N2B',
    'C1B': 'GSM4157071_Sample_9_C1B',
    'N5': 'GSM4157070_Sample_8_N5',
    'N1B': 'GSM4157072_Sample_10_N1B',
    'N4': 'GSM4157069_Sample_7_N4',
    'N4B': 'GSM4157076_Sample_14_N4B',
    'N5B': 'GSM4157078_Sample_16_N5B',
}
ada_ct['sample'] = ada_ct['Sample_Names(Library_Barcode)'].str.split('_').str[0]
ada_ct['seq'] = ada_ct['Sample_Names(Library_Barcode)'].str.split('_').str[1]
ada_ct = ada_ct.replace({"sample": ada_sample_dic})
ada_ct['barcode'] = ada_ct['sample'].str.cat(ada_ct['seq'], sep = "_") + '-1'
study_adata.obs = study_adata.obs.merge(ada_ct[['barcode', 'Level_1_cell_type','Level_2_cell_type']], 
                                        left_on='barcode', right_on='barcode', how='left')
study_adata.obs['authors_main_cell_type'] = study_adata.obs['Level_1_cell_type'].fillna(default_missing)
study_adata.obs['authors_sub_cell_type'] = study_adata.obs['Level_2_cell_type'].fillna(default_missing)
study_adata.obs['authors_main_cell_type'] = study_adata.obs['Level_1_cell_type'].fillna(default_missing)
study_adata.obs['authors_sub_cell_type'] = study_adata.obs['Level_2_cell_type'].fillna(default_missing)
study_adata.obs = study_adata.obs.drop('Level_1_cell_type', axis=1)
study_adata.obs = study_adata.obs.drop('Level_2_cell_type', axis=1)

In [7]:
study_adata.obs.head()

,barcode,study_name,sample_name,sample_source,sample_organism,sample_tissue_region,donor_gender,donor_age_years,cause_of_death,authors_main_cell_type,authors_sub_cell_type,Nurr_status
0,GSM4157067_Sample_5_C3_AAACCTGAGATACACA-1,Agarwal_NatCommun_2020_32826893,GSM4157067_Sample_5_C3,Brain,Homo sapiens,Cortex,Male,59,Unknown,Inhibitory neurons,In1a,Unknown
1,GSM4157067_Sample_5_C3_AAACCTGCACAGATTC-1,Agarwal_NatCommun_2020_32826893,GSM4157067_Sample_5_C3,Brain,Homo sapiens,Cortex,Male,59,Unknown,Excitatory neurons,Ex3,Unknown
2,GSM4157067_Sample_5_C3_AAACCTGCACGAAAGC-1,Agarwal_NatCommun_2020_32826893,GSM4157067_Sample_5_C3,Brain,Homo sapiens,Cortex,Male,59,Unknown,Microglia,Microglia,Unknown
3,GSM4157067_Sample_5_C3_AAACCTGGTAAATGAC-1,Agarwal_NatCommun_2020_32826893,GSM4157067_Sample_5_C3,Brain,Homo sapiens,Cortex,Male,59,Unknown,Excitatory neurons,Ex3,Unknown
4,GSM4157067_Sample_5_C3_AAACCTGGTACAAGTA-1,Agarwal_NatCommun_2020_32826893,GSM4157067_Sample_5_C3,Brain,Homo sapiens,Cortex,Male,59,Unknown,Inhibitory neurons,In4b,Unknown


#### Write standardized files

In [8]:
# Write zarr files
out_zarr = os.path.join(outdir_zarr, study_name + ".zarr")
study_adata.write_zarr(store = out_zarr, chunks=[study_adata.shape[0], 5])

# Write h5ad file
os.makedirs(os.path.join(outdir, 'adata'), exist_ok=True)
out_h5ad = os.path.join(outdir, 'adata', study_name + '.h5ad')
study_adata.write(out_h5ad, compression='gzip')

... storing 'study_name' as categorical
... storing 'sample_name' as categorical
... storing 'sample_source' as categorical
... storing 'sample_organism' as categorical
... storing 'sample_tissue_region' as categorical
... storing 'donor_gender' as categorical
... storing 'donor_age_years' as categorical
... storing 'cause_of_death' as categorical
... storing 'authors_main_cell_type' as categorical
... storing 'authors_sub_cell_type' as categorical
... storing 'Nurr_status' as categorical


## Standardize Kamath_2022 (from CELLxGENE h5ad files)
Note: an Upset plot analysis of barcodes across the h5ad files: `~/sc_and_spatial_parkinson/kamath_barcodes/LISTS_AND_COMMANDS/gets_upsetplot.sh` showed that the Nurr-Negative and Nurr-Positive files have barcodes distributed across the remaining files, and hence they weren't processed here.

#### Study parameters

In [9]:
study_name_ids = "Kamath_NatNeurosci_2022_35513515_CELLxGENE_IDs"
study_name = "Kamath_NatNeurosci_2022_35513515"
sex_dic = {
    'female': 'Female',
    'male': 'Male',
}
exclude_datasets = ['Nurr_negative', 'Nurr_positive']

input_sm_ids = (f"https://docs.google.com/spreadsheets/d/{google_id_sample_metadata}/gviz/tq?tqx=out:csv&sheet={study_name_ids}")
study_df = pd.read_csv(io.BytesIO(requests.get(input_sm_ids).content), dtype=str)
study_df.info(verbose=False)

indir_orig_h5ad = os.path.expanduser('~/MEASUREMENTS/HUMAN/Kamath_NatNeurosci_2022_35513515/FROM_CELLXGENE/ORIGINAL_FILES/')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Columns: 7 entries, cellxgene_dataset_id to Comments
dtypes: object(7)
memory usage: 636.0+ bytes


#### Standardize metadata and downsample barcodes for downstream analysis

In [10]:
conc_adatas = []
for idx, fields in study_df.iterrows():
    cxg_id = fields.iloc[0]
    cxg_name = fields.iloc[1]
    main_ct  = fields.iloc[2]
    downsample_to = int(fields.iloc[5])
    logger.info((f"{cxg_id} {cxg_name}"))
    orig_h5ad = os.path.join(indir_orig_h5ad, cxg_id + '.h5ad')
    adata = sc.read_h5ad(orig_h5ad)
    if main_ct in exclude_datasets:
        if main_ct == "Nurr_negative":
            Nurr_negative_ls = list(adata.obs.index)
        if main_ct == "Nurr_positive":
            Nurr_positive_ls = list(adata.obs.index)
        logger.info((f"\n{cxg_name}\nBefore: {adata.shape}\nNot downsampled"))
    else:
        # Standardize barcode IDs and metadata
        adata.obs['barcode'] = adata.obs.index
        adata.obs['sample_name'] = adata.obs.index.str.split('_', n=1).str[0]
        adata.obs['study_name'] = study_name
        adata.obs['donor_age_years'] = adata.obs['development_stage'].str.replace('-year-old stage','')
        adata.obs['donor_gender'] = adata.obs.replace({'sex': sex_dic})['sex']
        adata.obs['sample_organism'] = adata.obs['organism']
        adata.obs['sample_source'] = 'Brain'
        adata.obs['sample_tissue_region'] = adata.obs['tissue']
        adata.obs['authors_sub_cell_type'] = adata.obs['author_cell_type']
        adata.obs['cause_of_death'] = adata.obs['author_Cause_of_Death']
        adata.obs['authors_main_cell_type'] = main_ct
        
        # Downsample barcodes for downstream analysis
        adata_sub = sc.pp.sample(adata, axis='obs', n=downsample_to, copy=True, rng=42)
        conc_adatas.append(adata_sub)
        logger.info((f"\n{cxg_name}\nBefore: {adata.shape}\nAfter: {adata_sub.shape}"))

INFO:__main__:ac189051-14d0-4aae-a181-dbcfef4943f7 Human Oligodendrocytes 10x scRNA-seq
INFO:__main__:
Human Oligodendrocytes 10x scRNA-seq
Before: (178815, 34010)
After: (20000, 34010)
INFO:__main__:a381924d-10ec-42c9-a8b0-3afd2e753b40 Human Nurr-Negative Nuclei 10x scRNA-seq
INFO:__main__:
Human Nurr-Negative Nuclei 10x scRNA-seq
Before: (104097, 34010)
Not downsampled
INFO:__main__:f833b909-7acd-4ad6-b02e-c9a132c5829c Human Non-DA Neurons 10x scRNA-seq
INFO:__main__:
Human Non-DA Neurons 10x scRNA-seq
Before: (91479, 34010)
After: (20000, 34010)
INFO:__main__:f16f0630-2999-476e-909d-4ccddd238385 Human Nurr-Positive Nuclei 10x scRNA-seq
INFO:__main__:
Human Nurr-Positive Nuclei 10x scRNA-seq
Before: (80576, 34010)
Not downsampled
INFO:__main__:37d5bb98-6565-43a6-9a1a-c95136ca926a Human Astrocytes 10x scRNA-seq
INFO:__main__:
Human Astrocytes 10x scRNA-seq
Before: (33506, 34010)
After: (10000, 34010)
INFO:__main__:618ed998-f8f4-478c-b209-2f678aea6449 Human Human Microglia 10x scRNA-se

#### Concatenate cell type specific AnnData objects into a single one

In [11]:
study_adata = conc_adatas[0].concatenate(conc_adatas[1:], join="inner", index_unique=None)
Nurr_positive_cells = list(study_adata.obs[study_adata.obs['barcode'].isin(Nurr_positive_ls)].index)
Nurr_negative_cells = list(study_adata.obs[study_adata.obs['barcode'].isin(Nurr_negative_ls)].index)
study_adata.obs.loc[study_adata.obs.index.isin(Nurr_positive_cells), 'Nurr_status'] = "pos"
study_adata.obs.loc[study_adata.obs.index.isin(Nurr_negative_cells), 'Nurr_status'] = "neg"
study_adata.obs['Nurr_status'] = study_adata.obs['Nurr_status'].fillna(default_missing)
study_adata.obs = study_adata.obs[default_sample_metadata_cols + extra_sample_metadata_cols]

# Uppercase gene names and remove ENSG* code
study_adata.var['gene'] = study_adata.var['feature_name'].str.replace(r'_ENSG[0-9]+','',regex=True).apply(lambda x: x.upper())
logger.info((f"\nFinal: {study_adata}"))

INFO:__main__:
Final: AnnData object with n_obs × n_vars = 103691 × 34010
    obs: 'barcode', 'study_name', 'sample_name', 'sample_source', 'sample_organism', 'sample_tissue_region', 'donor_gender', 'donor_age_years', 'cause_of_death', 'authors_main_cell_type', 'authors_sub_cell_type', 'Nurr_status'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'gene'
    obsm: 'X_umap'


In [13]:
study_adata.obs.head()

,barcode,study_name,sample_name,sample_source,sample_organism,sample_tissue_region,donor_gender,donor_age_years,cause_of_death,authors_main_cell_type,authors_sub_cell_type,Nurr_status
index,,,,,,,,,,,,
pPDsHSrSNxi6173d200429DAPIA_GTGAGGAAGGGAGGGT-1,pPDsHSrSNxi6173d200429DAPIA_GTGAGGAAGGGAGGGT-1,Kamath_NatNeurosci_2022_35513515,pPDsHSrSNxi6173d200429DAPIA,Brain,Homo sapiens,substantia nigra pars compacta,Male,49,Not listed,Oligodendrocytes,Olig_PLXDC2,neg
pPDsHSrSNxi3482d200429DAPIB_GAACACTAGCGCCATC-1,pPDsHSrSNxi3482d200429DAPIB_GAACACTAGCGCCATC-1,Kamath_NatNeurosci_2022_35513515,pPDsHSrSNxi3482d200429DAPIB,Brain,Homo sapiens,substantia nigra pars compacta,Female,79,Coronary artery disease,Oligodendrocytes,Olig_PLXDC2_KCNAB1,neg
pPDsHSrSNxi3482d200429DAPIB_CTCCATGGTATAATGG-1,pPDsHSrSNxi3482d200429DAPIB_CTCCATGGTATAATGG-1,Kamath_NatNeurosci_2022_35513515,pPDsHSrSNxi3482d200429DAPIB,Brain,Homo sapiens,substantia nigra pars compacta,Female,79,Coronary artery disease,Oligodendrocytes,Olig_PLXDC2,neg
pPDsHSrSNxi3345d200429DAPIA_CTTTCAAGTAGACACG-1,pPDsHSrSNxi3345d200429DAPIA_CTTTCAAGTAGACACG-1,Kamath_NatNeurosci_2022_35513515,pPDsHSrSNxi3345d200429DAPIA,Brain,Homo sapiens,substantia nigra pars compacta,Female,82,Colon cancer,Oligodendrocytes,Olig_PLXDC2_SFRP1,neg
pPDsHSrSNxi3345d200429DAPIB_AGGTTACAGATGGTCG-1,pPDsHSrSNxi3345d200429DAPIB_AGGTTACAGATGGTCG-1,Kamath_NatNeurosci_2022_35513515,pPDsHSrSNxi3345d200429DAPIB,Brain,Homo sapiens,substantia nigra pars compacta,Female,82,Colon cancer,Oligodendrocytes,Olig_PLXDC2_SFRP1,neg


#### Write standardized files

In [14]:
# Write zarr files
out_zarr = os.path.join(outdir_zarr, study_name + ".zarr")
study_adata.write_zarr(store = out_zarr, chunks=[study_adata.shape[0], 5])

# Write h5ad file
os.makedirs(os.path.join(outdir, 'adata'), exist_ok=True)
out_h5ad = os.path.join(outdir, 'adata', study_name + '.h5ad')
study_adata.write(out_h5ad, compression='gzip')

... storing 'study_name' as categorical
... storing 'sample_name' as categorical
... storing 'sample_source' as categorical
... storing 'donor_age_years' as categorical
... storing 'authors_main_cell_type' as categorical
... storing 'Nurr_status' as categorical
... storing 'gene' as categorical


## Evaluate gene ID overlap between studies

In [15]:
study_names = ['Agarwal_NatCommun_2020_32826893','Kamath_NatNeurosci_2022_35513515']
conc_adatas = []
for study_name in study_names:
    logger.info((f"Read: {study_name}"))
    adata = convert_zarr_to_adata(os.path.join(outdir_zarr, study_name + '.zarr'))
    logger.info((f"Read: {adata}"))
    conc_adatas.append(adata)
adata = conc_adatas[0].concatenate(conc_adatas[1:], join="inner", index_unique=None)
logger.info((f"Concatenated: {adata}"))

INFO:__main__:Read: Agarwal_NatCommun_2020_32826893
INFO:__main__:Read: AnnData object with n_obs × n_vars = 18120 × 33694
    obs: 'barcode', 'study_name', 'sample_name', 'sample_source', 'sample_organism', 'sample_tissue_region', 'donor_gender', 'donor_age_years', 'cause_of_death', 'authors_main_cell_type', 'authors_sub_cell_type', 'Nurr_status'
    var: 'gene'
INFO:__main__:Read: Kamath_NatNeurosci_2022_35513515
INFO:__main__:Read: AnnData object with n_obs × n_vars = 103691 × 34010
    obs: 'barcode', 'study_name', 'sample_name', 'sample_source', 'sample_organism', 'sample_tissue_region', 'donor_gender', 'donor_age_years', 'cause_of_death', 'authors_main_cell_type', 'authors_sub_cell_type', 'Nurr_status'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type', 'gene'
    obsm: 'X_umap'
INFO:__main__:Concatenated: AnnData object with n_obs × n_vars = 121811 × 29546
    obs: 'barcode', 'study_name', 'sample_name', 'samp

Note, the vast majority of differences in gene ID's between the  two studies are lncRNA (9,274 out of 10,448). 1,057 protein_coding ID's would need to be mapped using aliases (not done here)

In [16]:
# All types of features
counts = adata.var['feature_type-1'].value_counts(dropna=False)
counts[counts > 0]

feature_type-1
protein_coding                        17819
lncRNA                                11197
IG_V_pseudogene                         103
IG_V_gene                                88
transcribed_unprocessed_pseudogene       85
TR_J_gene                                63
TR_V_gene                                58
transcribed_unitary_pseudogene           45
transcribed_processed_pseudogene         21
TR_V_pseudogene                          20
IG_J_gene                                12
IG_C_pseudogene                           6
IG_D_gene                                 6
IG_C_gene                                 5
TR_J_pseudogene                           4
unprocessed_pseudogene                    4
TR_C_gene                                 3
TR_D_gene                                 3
processed_pseudogene                      2
TEC                                       1
scRNA                                     1
Name: count, dtype: int64

In [17]:
# Features with different gene names (unmapped) between studies
counts = adata.var[adata.var['gene-0'] != adata.var['gene-1']]['feature_type-1'].value_counts(dropna=False)
counts[counts > 0]

feature_type-1
lncRNA                                9274
protein_coding                        1057
transcribed_unprocessed_pseudogene      60
transcribed_unitary_pseudogene          39
transcribed_processed_pseudogene        13
unprocessed_pseudogene                   2
processed_pseudogene                     2
TEC                                      1
Name: count, dtype: int64

## End of script